### Agents

In [9]:
import os,sys
sys.path.insert(0,'../../libs')
from utils import load_json

### Load all API keys 
openai_key = load_json('/home/chengyu.huang/project/Fund_projects/openai_key.json') 
os.environ['OPENAI_API_KEY'] = openai_key['ChatGPT']['API_KEY']

### initiate a llm

In [2]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],  # platform.openai.com
    temperature=0,
    model_name="gpt-3.5-turbo"
)

/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/chengyu.huang/anaconda3/envs/sbert/lib/python3.8/site-packages/langchain/llms/openai.py:739: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


#### Initialize a calculator tool using LLMMathChain

In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [4]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [5]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

#### intiate a zero shoot react agent to choose whcih tool to use 

In [6]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description",   ## for now, it doesn't have memory, only use description to do the filtering 
	tools=tools,
	llm=llm,
	verbose=True,        ## set it to false, we can get output directly 
	max_iterations=3     ## limit it to max loop in case it get stucked 
)

In [7]:
## give it a simple try 
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I need to use the calculator to solve this problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

- a slightly more complicated problem 

In [8]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
We need to convert the apple boxes to the number of apples they contain.
Action: Calculator
Action Input: 2.5 x 8
Observation: Answer: 20.0
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 5904a56c90a266cbb216338fc2cf6088 in your message.).


Now we add the number of apples Giorgio brought to the number of apples Mary had.
Action: Calculator
Action Input: 4 + 20
Observation: Answer: 24
Thought:The final answer is 24 apples. 
Final Answer: 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': '24 apples.'}

A more common case is we add a basic llm chain and plus all other tools, so that if additional tools are not useful, it will default to use llm 

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)
     

In [12]:
## add llm tool to tool list 
tools.append(llm_tool)

## initiate the zero shot agent again 
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)


In [13]:
## now try it out 
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
I don't know the answer to this question off the top of my head, so I should use a language model to search for it.
Action: Language Model
Action Input: "What is the capital of Norway?"
Observation: The capital of Norway is Oslo.
Thought:I now know the answer to the question.
Final Answer: Oslo is the capital of Norway.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'Oslo is the capital of Norway.'}

In [14]:
## give it a simple try 
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
I should use the calculator for this math problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the answer to the math problem.
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

#### There are a lot prebuilt tools in langchain

In [20]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [21]:
tools[0].name,tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

## What is an agent?
**Definition**: The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. Until now, we covered several building blocks in isolation. Let's see them come to life.

The official definition of agents is the following:


> Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

In [4]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [22]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.
Language Model: use this tool for general purpose queries and logic
Language Model: use this tool for general purpose queries and logic

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator, Language Model, Language Model]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### see [here](https://github.com/pinecone-io/examples/blob/master/generation/langchain/handbook/06-langchain-agents.ipynb) for an example of combining sql agent together with calculator to perform a task

Suffice it to say for now that **the LLM now has the ability to 'reason' on how to best use tools** to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to [this](https://arxiv.org/pdf/2205.00445.pdf) paper. 

Finally, let's pay attention to the 'agent_scratchpad'. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

### Agent type #2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [25]:
result = count_tokens(
    conversational_agent, 
    "Please provide me the stock prices for ABC on January the 1st"
                     )

## we did not add db query tool here, so it won't work. 



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: I'm sorry, but I don't have access to real-time stock prices. However, you can check financial news websites or use a stock market app to get the latest stock prices for ABC on January 1st.

> Finished chain.
Spent a total of 446 tokens


In [26]:
### the agent will have a memory history 
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

### Agent type #3: React Docstore

- This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.
- With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [42]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]
## initiate a docstore agent 
docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

In [43]:
count_tokens(docstore_agent, "Where is George Washington's hometown?")   ##



> Entering new AgentExecutor chain...
Thought: I need to search for George Washington's hometown and find its location.
Action: Search[George Washington's hometown]
Observation: Presidents' Day, officially Washington's Birthday at the federal governmental level, is a holiday in the United States celebrated on the third Monday of February. It is often celebrated to honor all those who served as presidents of the United States and, since 1879, has been the federal holiday honoring Founding Father George Washington, who led the Continental Army to victory in the American Revolutionary War, presided at the Constitutional Convention of 1787, and was the first U.S. president.
The day is an official state holiday in most states, with names including Washington's Birthday, Presidents' Day, President's Day, Presidents Day, and Washington's and Lincoln's Birthday. The various states use 15 different names. Depending upon the specific law, the state holiday may officially celebrate Washington a

{'input': "Where is George Washington's hometown?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [44]:
### the agent will have a docstroe agent 
print(docstore_agent.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

- In short, it contains several examples of the `Question` > `Thought` > `Action` > `Observation` loop, that include the `Search` and `Lookup` tools.
- If you want to learn more about this approach [this](https://arxiv.org/pdf/2210.03629.pdf) is the paper for ReAct

### Agent type #4: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer. 

In [19]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = GoogleSerperAPIWrapper(serper_api_key=openai_key['Serper']['API_KEY'])
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [20]:
search.run("Obama's first name?")

'Barack Hussein Obama II'

In [21]:
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")



> Entering new AgentExecutor chain...


AuthenticationError: <empty message>

In [6]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin